In [2]:
import re
from bs4 import BeautifulSoup
import xlwt
import urllib

def main():
    baseurl='https://movie.douban.com/top250?start='
    datalist=getData(baseurl)
    savepath=r'/Users/m/Desktop/(自己练习)豆瓣电影top250榜单.xls'
    saveData(datalist,savepath)

findLink=re.compile(r'<a href="(.*?)">',re.S)
findImg=re.compile(r'<img.*src="(.*?)"',re.S)
findTitle=re.compile(r'<span class="title">(.*?)</span>',re.S)
findRate=re.compile(r'<span class="rating_num" property="v:average">(.*?)</span>',re.S)
findRnum=re.compile(r'<span>(\d*)人评价</span>',re.S)                #  正则表达式里的数字筛选是\d, *重复n次
findQuo=re.compile(r'<span class="inq">(.*?)</span>',re.S)
findBd=re.compile(r'<p class="">(.*?)</p>',re.S)


def getData(baseurl):
    data=[]
    for i in range(0,10):
        url=baseurl+str(i*25)
        html=askURL(url)
        #   print(html)               #测试用
        soup=BeautifulSoup(html,'html.parser')      #   先创建一个BeautifulSoup对象，指定解码器
        for item in soup.find_all('div',class_='item'):
            datalist=[]                                          #注意结构：所有的电影信息是一大块，里面的每个电影的所有信息是另一块
            item=str(item)                                    # 记得把提取到的item字符串化才能re处理
            Link=re.findall(findLink,item)[0]
            datalist.append(Link)
            Img=re.findall(findImg,item)[0]
            datalist.append(Img)
            Title=re.findall(findTitle,item)
            if (len(Title)==2):                                   
                ctitle=Title[0]
                datalist.append(ctitle)
                otitle=Title[1].replace('/','')
                datalist.append(otitle)
            else:
                datalist.append(Title)
                datalist.append(' ')
            Rate=re.findall(findRate,item)[0]
            datalist.append(Rate)
            Rnum=re.findall(findRnum,item)[0]
            datalist.append(Rnum)
            Quo=re.findall(findQuo,item)
            if len(Quo)!=0:
                quo=Quo[0].replace('。','')        #列表格式没有.replace方法，要先用[ ]确定列表中的哪一项
                datalist.append(quo)
            else:
                datalist.append(' ')
            Bd=re.findall(findBd,item)[0]
            Bd=re.sub(' ','',Bd)
            Bd=re.sub(r'&nbsp;',' ',Bd)
            Bd=re.sub(r'<br>','',Bd)
            Bd=re.sub(r'<br/>','',Bd)
            datalist.append(Bd)
            data.append(datalist)           # 注意结构：先把每个电影各自的所有信息保存，之后再把所有电影的各自信息进行整合
    return data
#     print(datalist)                        测试用
        

def askURL(baseurl):
    html=''
    head={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:69.0) Gecko/20100101 Firefox/69.0'}
    content=urllib.request.Request(baseurl,headers=head)
    get=urllib.request.urlopen(content)
    html=get.read().decode('utf-8')
    return html

def saveData(datalist,savepath):
    print('save...')
    book=xlwt.Workbook(encoding='utf-8',style_compression=0)
    sheet=book.add_sheet('豆瓣电影Top250榜单',cell_overwrite_ok=True)
    col=('电影链接','电影图片','电影中文名','电影外文名','电影评分','评价人数','高分短评','电影详情')
    for i in range(0,8):
        sheet.write(0,i,col[i])
    for i in range(0,250):
        print('第%d条'%(i+1))
        data=datalist[i]            #把所有电影缩小化到每个电影的范围上
        for j in range(0,8):
            sheet.write(i+1,j,data[j])
#     for data in datalist:
#         for i in range(0,8):
#             sheet.write(int(data)+1,i,data[i])
    book.save(savepath)
    
        


if __name__=='__main__':
    main()
    print('爬取完毕')
    
    
    
    
    
    

save...


TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'